# 03_Modeling

https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.IsolationForest.html

## Imports

In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import roc_auc_score, average_precision_score
import joblib
import plotly.express as px
from sklearn.decomposition import PCA

## Load data

In [15]:
df = pd.read_csv('../data/preprocessed_datasets/windowed_dataset_cleaned.csv')
X = df[df.columns.difference(['is_attack'])]  # Features
y = df['is_attack']  # Labels

## Modeling

In [16]:
contamination = y.mean()
print(f"Contamination (proporción de ataques): {contamination:.4f}")
from sklearn.ensemble import IsolationForest

from sklearn.pipeline import Pipeline
model = Pipeline([
    ('scaler', RobustScaler()),  # Escala a media=0, std=1
    ('if', IsolationForest(
        n_estimators=100,
        contamination=contamination, # cantidad estimada de outliers, donde pone el threeshold
        max_samples='auto', # number of samples to draw from X to train each base estimator
        random_state=42,
        n_jobs=-1 # usar todos los núcleos disponibles
    ))
])

model.fit(X)

Contamination (proporción de ataques): 0.0144


,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('scaler', ...), ('if', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"with_centering with_centering: bool, default=TrueIf `True`, center the data before scaling.This will cause :meth:`transform` to raise an exception when attemptedon sparse matrices, because centering them entails building a densematrix which in common use cases is likely to be too large to fit inmemory.",True
,"with_scaling with_scaling: bool, default=TrueIf `True`, scale the data to interquartile range.",True
,"quantile_range quantile_range: tuple (q_min, q_max), 0.0 < q_min < q_max < 100.0, default=(25.0, 75.0)Quantile range used to calculate `scale_`. By default this is equal tothe IQR, i.e., `q_min` is the first quantile and `q_max` is the thirdquantile... versionadded:: 0.18","(25.0, ...)"
,"copy copy: bool, default=TrueIf `False`, try to avoid a copy and do inplace scaling instead.This is not guaranteed to always work inplace; e.g. if the data isnot a NumPy array or scipy.sparse CSR matrix, a copy may still bereturned.",True
,"unit_variance unit_variance: bool, default=FalseIf `True`, scale data so that normally distributed features have avariance of 1. In general, if the difference between the x-values of`q_max` and `q_min` for a standard normal distribution is greaterthan 1, the dataset will be scaled down. If less than 1, the datasetwill be scaled up... versionadded:: 0.24",False
,"n_estimators n_estimators: int, default=100The number of base estimators in the ensemble.",100
,"max_samples max_samples: ""auto"", int or float, default=""auto""The number of samples to draw from X to train each base estimator.- If int, then draw `max_samples` samples.- If float, then draw `max_samples * X.shape[0]` samples.- If ""auto"", then `max_samples=min(256, n_samples)`.If max_samples is larger than the number of samples provided,all samples will be used for all trees (no sampling).",'auto'


Isolation forest is based on underlying Random Decision Trees that are going to isolate each sample from the rest of samples. The easier it is to isolate the sample the high anomaly score it will get.

- The **n_estimator** is the number of decision trees that are going to be used. The default value was chosen.
- The main hyperparameter of this model is **contamination**. This parameters define where is going to put the threshold the model to tell if a sample is an anomaly or not. It won't influence its training or the anomaly score it is going to give to each sample. We decides to put the proportion of attacks the data has to see if it if able to find them within the data

In [17]:
df['anomaly_score'] = -model.score_samples(X)
df['anomaly_label'] = model.predict(X) == -1
print(roc_auc_score(y, df['anomaly_score']))
print(average_precision_score(y, df['anomaly_score']))

0.9997102374855454
0.9906531195898469


- These metric show that the model captures nearly every attack

In [18]:
df.sort_values('anomaly_score', ascending=False).head(5)

,n_connections,id.orig_p_mean,id.orig_p_std,id.orig_p_max,id.resp_p_std,orig_bytes_mean,orig_bytes_std,orig_bytes_max,resp_bytes_mean,resp_bytes_std,...,recent_activity_score_max,recent_docker_event_mean,recent_docker_event_std,recent_docker_event_max,time_since_container_start_mean,time_since_container_start_std,time_since_container_start_max,is_attack,anomaly_score,anomaly_label
326,7,39327.142857,17.620335,39358,0.0,837.142857,85.407706,1029.0,6333.714286,4363.623789,...,1.0,0.0,0.0,0,0.0,0.0,0.0,1,0.763191,True
325,7,39327.142857,17.620335,39358,0.0,837.142857,85.407706,1029.0,6333.714286,4363.623789,...,1.0,0.0,0.0,0,0.0,0.0,0.0,1,0.763191,True
324,7,39327.142857,17.620335,39358,0.0,837.142857,85.407706,1029.0,6333.714286,4363.623789,...,1.0,0.0,0.0,0,0.0,0.0,0.0,1,0.763191,True
76,7,47967.142857,13.606721,47982,0.0,837.000000,85.033327,1028.0,6330.142857,4364.915899,...,1.0,0.0,0.0,0,0.0,0.0,0.0,1,0.762414,True
77,7,47967.142857,13.606721,47982,0.0,837.000000,85.033327,1028.0,6330.142857,4364.915899,...,1.0,0.0,0.0,0,0.0,0.0,0.0,1,0.762414,True


In [19]:
df.sort_values('anomaly_score').head(10)

,n_connections,id.orig_p_mean,id.orig_p_std,id.orig_p_max,id.resp_p_std,orig_bytes_mean,orig_bytes_std,orig_bytes_max,resp_bytes_mean,resp_bytes_std,...,recent_activity_score_max,recent_docker_event_mean,recent_docker_event_std,recent_docker_event_max,time_since_container_start_mean,time_since_container_start_std,time_since_container_start_max,is_attack,anomaly_score,anomaly_label
20254,1,50797.0,0.0,50797,0.0,796.0,0.0,796.0,1737.1,0.0,...,0.305274,1.0,0.0,1,5.556765,0.0,5.556765,0,0.387619,False
8367,1,44923.0,0.0,44923,0.0,1069.9,0.0,1069.9,1744.1,0.0,...,0.260883,1.0,0.0,1,6.903722,0.0,6.903722,0,0.388321,False
8366,1,44923.0,0.0,44923,0.0,1069.9,0.0,1069.9,1744.1,0.0,...,0.260883,1.0,0.0,1,6.903722,0.0,6.903722,0,0.388321,False
8365,1,44923.0,0.0,44923,0.0,1069.9,0.0,1069.9,1744.1,0.0,...,0.260883,1.0,0.0,1,6.903722,0.0,6.903722,0,0.388321,False
3943,1,50857.0,0.0,50857,0.0,800.9,0.0,800.9,1740.4,0.0,...,0.204833,0.0,0.0,0,7.044026,0.0,7.044026,0,0.388934,False
3945,1,50857.0,0.0,50857,0.0,800.9,0.0,800.9,1740.4,0.0,...,0.204833,0.0,0.0,0,7.044026,0.0,7.044026,0,0.388934,False
3944,1,50857.0,0.0,50857,0.0,800.9,0.0,800.9,1740.4,0.0,...,0.204833,0.0,0.0,0,7.044026,0.0,7.044026,0,0.388934,False
8076,1,56282.0,0.0,56282,0.0,1066.6,0.0,1066.6,1740.3,0.0,...,0.305080,1.0,0.0,1,8.031483,0.0,8.031483,0,0.389638,False
8077,1,56282.0,0.0,56282,0.0,1066.6,0.0,1066.6,1740.3,0.0,...,0.305080,1.0,0.0,1,8.031483,0.0,8.031483,0,0.389638,False
8078,1,56282.0,0.0,56282,0.0,1066.6,0.0,1066.6,1740.3,0.0,...,0.305080,1.0,0.0,1,8.031483,0.0,8.031483,0,0.389638,False


## Visualize results

In [20]:
pca_pipeline = Pipeline([
    ('scaler', RobustScaler()),
    ('pca' , PCA(n_components=2, random_state=42))

])
X_pca = pca_pipeline.fit_transform(X)

print("Varianza explicada por componente:")
print(pca_pipeline.__getitem__('pca').explained_variance_ratio_)
print("Varianza total explicada:", pca_pipeline.__getitem__('pca').explained_variance_ratio_.sum())

# Crear DataFrame para visualización
df_pca_vis = pd.DataFrame({
    'PCA1': X_pca[:, 0],
    'PCA2': X_pca[:, 1],
    'anomaly_label': df['anomaly_label'].map({True: 'Anomaly', False: 'Normal'}),
    'is_attack': y
})

# Gráfico interactivo con Plotly
fig = px.scatter(
    df_pca_vis,
    x='PCA1',
    y='PCA2',
    color='anomaly_label',
    title="PCA (2D) – Isolation Forest Anomaly Detection",
    width=900,
    height=700,
    hover_data=['is_attack'],
    color_discrete_map={'Anomaly': 'red', 'Normal': 'blue'}
)

fig.update_traces(marker=dict(size=6, opacity=0.8))
fig.update_layout(legend_title_text='Predicted Label')
fig.show()

Varianza explicada por componente:
[0.55767164 0.43215248]
Varianza total explicada: 0.9898241214491248


In [ ]:
import umap
import plotly.express as px

# -----------------------------
# UMAP embedding
# -----------------------------
umap_pipeline = Pipeline([
    ('scaler',RobustScaler()),
    ('umap',
        umap.UMAP(
        n_neighbors=30,
        min_dist=0.1,
        n_components=2,
        random_state=42
    ))
])

X_umap = umap_pipeline.fit_transform(X)

# Crear un DataFrame para la visualización
df_plot = pd.DataFrame({
    'UMAP1': X_umap[:, 0],
    'UMAP2': X_umap[:, 1],
    'anomaly_score': df['anomaly_score']
})

# Opcional: incluir más columnas originales para mostrar en el hover
# Por ejemplo, si quieres mostrar las primeras 5 características:
feature_cols = [
    'conn_count_10s_max',
    'burst_score_max',
    'bytes_ratio_mean',
    'recent_activity_score_std',
    'is_attack'
]
# Asegurar que 'is_attack' esté incluido

df_plot = pd.concat([df_plot, df[feature_cols].reset_index(drop=True)], axis=1)

# Crear gráfico interactivo
fig = px.scatter(
    df_plot,
    x='UMAP1',
    y='UMAP2',
    color='anomaly_score',
    color_continuous_scale='viridis',
    hover_data=feature_cols,  # ¡esto muestra los valores al pasar el ratón!
    title="UMAP projection – colored by anomaly score",
    width=900,
    height=700
)

fig.update_traces(marker=dict(size=6, opacity=0.8))
fig.show()

/Users/taxi/miniconda3/envs/consoning/lib/python3.11/site-packages/umap/umap_.py:1952: UserWarning:

n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.

/Users/taxi/miniconda3/envs/consoning/lib/python3.11/site-packages/sklearn/manifold/_spectral_embedding.py:324: UserWarning:

Graph is not fully connected, spectral embedding may not work as expected.



- The model gives to each sample an anomaly score. The samples central cluster have a low anomaly score with means that are difficult to separe from the rest of the samples.
- The island on the right is formed by a mid anomaly score which suggests that those samples are easier to isolate than the previous ones. As we have seen previously, those sample have a higher network intesity, connexions, etc. (try hover on those samples)
- If we zoom in the bottom right corner, we can see that there are two long islands and the model is actually able to make a difference between them in terms of anomaly score, both been more anomalous than the rest.

## Save Model

In [22]:
joblib.dump(model, "../models/isolation_forest_model.joblib")

['../models/isolation_forest_model.joblib']